# Obtaining Data

- Palmitoylation site information is obtained from SwissPalm
- sites.txt is obtained and convert to sites.csv
- sites.csv is saved at https://github.com/sonluongvu/Palm_structure

Open sites.csv

In [5]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv'

--2022-01-24 23:37:36--  https://github.com/sonluongvu/Palm_structure/raw/main/sites.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/sites.csv [following]
--2022-01-24 23:37:37--  https://raw.githubusercontent.com/sonluongvu/Palm_structure/main/sites.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3260327 (3.1M) [text/plain]
Saving to: ‘sites.csv.1’

sites.csv.1         100%[===================>]   3.11M  --.-KB/s    in 0.03s   

2022-01-24 23:37:37 (105 MB/s) - ‘sites.csv.1’ saved [3260327/3260327]



In [6]:
import pandas as pd
import numpy as np
sites_path = '/content/sites.csv'
sites = pd.read_csv(sites_path)
sites.shape

(7452, 26)

Number of palmitoylation sites based on organisms

In [7]:
sites.organism.value_counts()

Mus musculus                                                            4328
Homo sapiens                                                            2691
Rattus norvegicus                                                        160
Saccharomyces cerevisiae (strain ATCC 204508 / S288c)                     46
Bos taurus                                                                38
Arabidopsis thaliana                                                      30
Oryctolagus cuniculus                                                     30
Torpedo californica                                                       13
Drosophila melanogaster                                                   12
Hepatitis E virus genotype 3 (isolate Human/United States/US2)             8
Canis familiaris                                                           8
Semliki forest virus                                                       8
Sindbis virus                                                              8

- Most of the provided palmitoylation sites above are not verified by experiments
- Listed of verified sites ID is obtained from SwissPalm database

Obtain list of verified palm sites for Homo sapiens

In [8]:
! wget 'https://github.com/sonluongvu/Palm_structure/raw/main/Mus_musculus/palmprot_by_organism_targeted_sites_2.txt'

--2022-01-24 23:37:56--  https://github.com/sonluongvu/Palm_structure/raw/main/Homo_sapiens_verified_sites.csv
Resolving github.com (github.com)... 140.82.121.3
Connecting to github.com (github.com)|140.82.121.3|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-01-24 23:37:56 ERROR 404: Not Found.



In [9]:
Mus_musculus_id_path = '/content/palmprot_by_organism_targeted_sites_2.txt'
Mus_musculus_id = pd.read_csv(Homo_sapiens_id_path)
Mus_musculus_id.shape

(259, 2)

Obtain the list of id

In [10]:
Mus_musculus_list = []
for i in range(len(Mus_musculus_id.index)):
  x = str(Mus_musculus_id.iloc[i][0])
  y = 'SPALMS'+ x[7:]
  Mus_musculus_list.append(y)

len(Mus_musculus_list)

259

Obtain info for Homo_sapiens_list from sites

In [11]:
Mus_musculus_sites = sites[sites['id'].isin(Mus_musculus_list)]
Mus_musculus_sites.uniprot_ac.value_counts()

P56818    12
P06240     8
P60766     7
Q80YE4     6
Q3TBT3     6
          ..
Q0QWG9     1
O08858     1
O08917     1
Q9Z2W9     1
Q62226     1
Name: uniprot_ac, Length: 83, dtype: int64

Obtain protein sequence

In [12]:
! pip install biopython

     |████████████████████████████████| 2.3 MB 12.8 MB/s 


In [13]:
import requests as r
from Bio import SeqIO
from io import StringIO 

Uniprot_ID = Mus_musculus_sites['uniprot_ac']
Seq_list = []
for cID in Uniprot_ID:
  baseUrl="http://www.uniprot.org/uniprot/"
  currentUrl=baseUrl+cID+".fasta"
  response = r.post(currentUrl)
  cData=''.join(response.text)

  Seq=StringIO(cData)
  pSeq=list(SeqIO.parse(Seq,'fasta'))
  Seq_list.append(pSeq)

In [14]:
protein_seq_list = []
for record in Seq_list:
  sequence = 200*'-' + str(record[0].seq) +200*'-'
  protein_seq_list.append(sequence)
Mus_musculus_sites['Protein_seq'] = protein_seq_list
Mus_musculus_sites

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,id,hit_id,uniprot_ac,uniprot_id,uniprot_description,organism,main_gene_name,gene_names,study_id,study_organism,study_cell_type,study_subcellular_fraction,pmid,authors,title,year,hit_techniques,large_scale,hit_list,acyl_type,isoform,pos,uncertain_pos,pats,apts,site_techniques,Protein_seq
9,SPALMS25,SPALMH5911,P19785,ESR1_MOUSE,Estrogen receptor (ER) (ER-alpha) (Estradiol r...,Mus musculus,Esr1,"Nr3a1,Estra,Estr,Esr,Esr1",18,Cricetulus griseus,CHO-K1,Non-nuclear membrane,22031296,Pedram A et al.,DHHC-7 and -21 are palmitoylacyltransferases f...,2012,"Cleavage of thioester bonds with NH2OH,Cellula...",No,None,palmitate,NaN,451,No,"Q9D270,Q91WU6",NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
10,SPALMS28,SPALMH5913,P39688,FYN_MOUSE,Tyrosine-protein kinase Fyn (EC 2.7.10.2) (Pro...,Mus musculus,Fyn,Fyn,20,Chlorocebus aethiops,L Cells,Total cell lysate,7980442,Koegl M et al.,Palmitoylation of multiple Src-family kinases ...,1994,"Point mutation of substrate,Labeling of substr...",No,None,palmitate,NaN,3,No,NaN,NaN,NaN,----------------------------------------------...
11,SPALMS29,SPALMH5913,P39688,FYN_MOUSE,Tyrosine-protein kinase Fyn (EC 2.7.10.2) (Pro...,Mus musculus,Fyn,Fyn,20,Chlorocebus aethiops,L Cells,Total cell lysate,7980442,Koegl M et al.,Palmitoylation of multiple Src-family kinases ...,1994,"Point mutation of substrate,Labeling of substr...",No,None,palmitate,NaN,6,No,NaN,NaN,NaN,----------------------------------------------...
12,SPALMS31,SPALMH5915,P27600,GNA12_MOUSE,Guanine nucleotide-binding protein subunit alp...,Mus musculus,Gna12,"Gna-12,Gna12",22,Mus musculus,NIH-3T3,Total membrane,9485474,Jones TL et al.,Galpha12 requires acylation for its transformi...,1998,"Labeling of substrate with [3H]palmitic acid,P...",No,None,palmitate,NaN,11,No,NaN,NaN,"Point mutation of substrate,Labeling of substr...",----------------------------------------------...
13,SPALMS32,SPALMH5916,P21279,GNAQ_MOUSE,Guanine nucleotide-binding protein G(q) subuni...,Mus musculus,Gnaq,Gnaq,23,Homo sapiens,HEK293,Total cell lysate,8227063,Wedegaertner PB et al.,Palmitoylation is required for signaling funct...,1993,"Point mutation of substrate,Labeling of substr...",No,None,palmitate,NaN,10,No,NaN,NaN,NaN,----------------------------------------------...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7425,SPALMS5651,SPALMH26415,Q08460,KCMA1_MOUSE,Calcium-activated potassium channel subunit al...,Mus musculus,Kcnma1,"Kcnma,Kcnma1",291,Mus musculus,HEK293T,Plasma Membrane,22399288,Tian L et al.,Distinct acyl protein transferases and thioest...,2012,Cellular localization of substrate by fusion w...,No,None,palmitate,5.0,54,Yes,"A0PK84,Q5Y5T3","O75608,Q5VWZ2","Point mutation of substrate,Labeling of substr...",----------------------------------------------...
7426,SPALMS5652,SPALMH26415,Q08460,KCMA1_MOUSE,Calcium-activated potassium channel subunit al...,Mus musculus,Kcnma1,"Kcnma,Kcnma1",291,Mus musculus,HEK293T,Plasma Membrane,22399288,Tian L et al.,Distinct acyl protein transferases and thioest...,2012,Cellular localization of substrate by fusion w...,No,None,palmitate,5.0,56,Yes,"A0PK84,Q5Y5T3","O75608,Q5VWZ2","Point mutation of substrate,Labeling of substr...",----------------------------------------------...
7433,SPALMS5714,SPALMH26487,Q80YE4,LMTK1_MOUSE,Serine/threonine-protein kinase LMTK1 (EC 2.7....,Mus musculus,Aatk,"Lmtk1,Lmr1,Kiaa0641,Aatyk,Aatk",1087,Chlorocebus aethiops,COS7,Total membrane,18691334,Tsutsumi K et al.,Palmitoylation-dependent endosomal localizatio...,2008,Inhibition of palmitoylation with 2-bromopalmi...,No,None,palmitate,2.0,6,Yes,NaN,NaN,Labeling of substrate with [3H]palmitic acid,----------------------------------------------...
7434,SPALMS5778,SPALMH26611,Q08857,CD36_MOUSE,Platelet glycoprotein 4 (Glycoprotein IIIb) (G...,Mus musculus,Cd36,Cd36,1149,Mus musculus,Liver,Total cell lysate,2970524

In [15]:
columns = ['id', 'uniprot_ac', 'pos', 'protein_seq']
Mus_musculus_info = Mus_musculus_sites.filter(columns, axis = 1)
Mus_musculus_info['protein_seq'] = protein_seq_list
Mus_musculus_info.shape

(254, 4)

In [16]:
Mus_musculus_info.to_csv('Mus_musculus_info.csv', index = False)
type(Seq_list[1])

list

Obtain 100 aa upstream and 100 aa downstream